# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [111]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas as pd #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs

import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [13]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [9]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [10]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":818592713,"wgRevisionId":818592713,"wgArticleId":473317,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing cleanup from April 2008","All pages needing cleanup","Cleanup tagged articles without a reason field from April 2008","Wikipedia pages needing cleanup from April 2008","Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [14]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
(window.RLQ=window.RLQ||[]).push(functio


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [15]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [27]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

AttributeError: 'str' object has no attribute 'body'

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [17]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   \nContent analysis is a research method for st...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3                                                    
4                                                    
5   Content analysis is best understood as a broad...
6   The simplest and most objective form of conten...
7   A further step in analysis is the distinction ...
8   More generally, content analysis is research u...
9   By having contents of communication available ...
10  Robert Weber notes: "To make valid inferences ...
11  There are five types of texts in content analy...
12  Over the years, content analysis has been appl...
13  In recent times, particularly with the advent ...
14  Quantitative content analysis has enjoyed a re...
15  Recently, Arash Heydarian Pashakhanlou has arg...
16  Content analysis can also be described as stud...
17  The method of content an

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [18]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [19]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [20]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

## What the hell are pick-up artists talking about? A content analysis.

In [47]:
# scrape the sosuave.net message board/forum
djb_url = 'http://www.sosuave.net/forum/threads/most-commonly-asked-newbie-questions-answered-right-here.56520/'
djb_content = requests.get(djb_url)
djb_save = 'djb.html'
with open(djb_save, mode='w', encoding='utf-8') as f:
    f.write(djb_content.text)

In [161]:
# parse with beautifulsoup
with open(djb_save) as f:
    djb_soup = bs4.BeautifulSoup(f, 'html.parser')
    
# print all posts from the forum page
dj_content = djb_soup.body.findAll('article')

for x in dj_content:
	print(x.text)



					
					I have spanned these forums in search of the most commonly asked questions from not only newbies, but community members here. Check it out!


How can I tell if she likes me? 

The only way to be able to tell how interested she is in you is to make a move (number close, date request, kiss, etc.) Watch her reaction. If the move is unsuccessful the first time, you can try again later one more time. If it's unsuccessful the second time, ditch her and move on....in other words...NEXT.


How do I let a girl know how I feel? 

Make a move (number close, date request, kiss, etc.). DO NOT tell her with words how you feel or act like a suck-up and buy her gifts and dinners on the first date because you will come off way too eager and wussy. No, you need to be a challenge so that she can see you as being worth fighting for. When you approach a girl and you communicate with her for a little bit, get her email or number (or both) to show her that you like her enough to want to talk wit

In [66]:
# make a df of all forum posts
dj_posts = []
for post in dj_content:
    dj_posts.append(post.text)

postsDF = pd.DataFrame({'post' : dj_posts})
print(postsDF)

                                                 post
0   \n\n\t\t\t\t\t\n\t\t\t\t\tI have spanned these...
1   \n\nWhat do I do if she plays "hard to get"? \...
2   \n\n\t\t\t\t\t\n\t\t\t\t\tCrazyKid, I think we...
3   \n\n\t\t\t\t\t\n\t\t\t\t\tOr they could just r...
4   \n\n\n\nOriginally posted by Silquee Smoove \n...
5   \n\n\n\nAs do I. But first...It must go to Tip...
6   \n\n\n\nOriginally posted by Craig Reeves \n\n...
7   \n\n\t\t\t\t\t\n\t\t\t\t\t"Hey, I'm in a good ...
8   \n\n\n\nMe: So what do you do?\n\nHer: Oh, I'm...
9   \n\n\t\t\t\t\t\n\t\t\t\t\tI have placed it in ...
10  \n\n\t\t\t\t\t\n\t\t\t\t\tThanks for the advic...
11  \n\n\t\t\t\t\t\n\t\t\t\t\tOh thank you so much...
12  \n\n\n\nOriginally posted by Craig Reeves \nOh...
13  \n\n\n\nOriginally posted by Kourt \ny'all? mb...
14  \n\n\t\t\t\t\t\n\t\t\t\t\tWow. I am new to thi...
15  \n\n\t\t\t\t\t\n\t\t\t\t\tGood post especially...
16  \n\n\t\t\t\t\t\n\t\t\t\t\tAwesome post...\n\n\...
17  \n\n\n\nHow do I ask a g

In [241]:
postsDF

,post
0,\n\n\t\t\t\t\t\n\t\t\t\t\tI have spanned these...
1,"\n\nWhat do I do if she plays ""hard to get""? \..."
2,"\n\n\t\t\t\t\t\n\t\t\t\t\tCrazyKid, I think we..."
3,\n\n\t\t\t\t\t\n\t\t\t\t\tOr they could just r...
4,\n\n\n\nOriginally posted by Silquee Smoove \n...
5,\n\n\n\nAs do I. But first...It must go to Tip...
6,\n\n\n\nOriginally posted by Craig Reeves \n\n...
7,"\n\n\t\t\t\t\t\n\t\t\t\t\t""Hey, I'm in a good ..."
8,"\n\n\n\nMe: So what do you do?\n\nHer: Oh, I'm..."
9,\n\n\t\t\t\t\t\n\t\t\t\t\tI have placed it in ...


In [120]:
#find any numbers within the posts
findNum = re.compile(r'\d+')

for i in range(0, len(postsDF)):
	print(findNum.findall(postsDF.iloc[i,0]))

[]
['2', '4', '2', '3']
[]
[]
[]
[]
[]
[]
['5', '2']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [121]:
#find all mentions of man/men within the posts
findM = re.compile(r'm[ae]n', re.IGNORECASE)

for i in range(0, len(postsDF)):
	print(findM.findall(postsDF.iloc[i,0]))

['man', 'men', 'men', 'man', 'men', 'men', 'men', 'men', 'men']
['man', 'men', 'men', 'men', 'men', 'man', 'man', 'man', 'man', 'men', 'men', 'men', 'men', 'men']
[]
[]
[]
[]
[]
[]
[]
[]
['man']
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [122]:
#find all mentions of woman/women within the posts
findM = re.compile(r'wom[ae]n', re.IGNORECASE)

for i in range(0, len(postsDF)):
	print(findM.findall(postsDF.iloc[i,0]))

['woman', 'women', 'women', 'women', 'women', 'women', 'women', 'women']
['woman', 'Women']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


Now I want to try to find all the time-stamp data associated with the forum posts.

In [159]:
# the dates of the posts

post_date = djb_soup.findAll('span', {"class": "DateTime"})
for x in post_date:
	print(x.text)
    

Aug 8, 2004
Aug 8, 2004
Aug 8, 2004
Aug 8, 2004
Aug 8, 2004
Aug 8, 2004
Aug 8, 2004
Aug 8, 2004
Aug 8, 2004
Aug 9, 2004
Aug 9, 2004
Aug 10, 2004
Aug 10, 2004
Aug 10, 2004
Aug 10, 2004
Aug 11, 2004
Aug 12, 2004
Aug 12, 2004
Dec 3, 2004
Dec 3, 2004
Dec 12, 2004
Jul 4, 2005


In [147]:
# the join-dates of the posters

user_jd = djb_soup.findAll('dd')
for x in user_jd:
	print(x.text)
    
jds = []
for user in user_jd:
    jds.append(user.text)

jd_df = pd.DataFrame({'join_date' : jds})

Oct 18, 2003
519
14
Texas, USA
23
Oct 18, 2003
519
14
Texas, USA
23
Jul 12, 2003
725
1
Jax Bch, FL - 3rd Street Crew
26
Sep 1, 2003
856
0
Hell
769483.
Dec 7, 2001
5,519
6
22
Sep 1, 2003
856
0
Hell
769483.
Mar 27, 2003
523
1
Feb 7, 2004
422
2
Mi
20
Oct 18, 2003
519
14
Texas, USA
23
Nov 17, 2001
862
3
Swansea, Wales, UK
20
Aug 10, 2004
1
0
18
Oct 18, 2003
519
14
Texas, USA
23
Dec 7, 2001
5,519
6
22
Jun 2, 2004
204
0
Knoxville, TN
20
Aug 11, 2004
55
1
Europe
Too old
Feb 29, 2004
3,316
48
Baltimore, MD
29
Jun 6, 2004
95
0
BayArea, Cali
20
Sep 28, 2004
337
0
Oct 28, 2004
135
0
uknown
Jun 14, 2005
1,164
5
florida baby
17




						No, create an account now.

						Yes, my password is:


Forgot your password?





 Stay logged in


 Search titles only



Separate names with a comma.



 Search this thread only
 Search this forum only

 Display results as threads








Useful Searches



Recent Posts




More...

Default


In [157]:
keepDates = re.compile(r'\D{3,3}\d{2,2},', re.IGNORECASE)

for i in range(0, len(jd_df)):
	print(keepDates.findall(jd_df.iloc[i,0]))

['ct 18,']
[]
[]
[]
[]
['ct 18,']
[]
[]
[]
[]
['ul 12,']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ar 27,']
[]
[]
[]
[]
[]
[]
[]
['ct 18,']
[]
[]
[]
[]
['ov 17,']
[]
[]
[]
[]
['ug 10,']
[]
[]
[]
['ct 18,']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ug 11,']
[]
[]
[]
[]
['eb 29,']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['ep 28,']
[]
[]
['ct 28,']
[]
[]
[]
['un 14,']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [164]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning'), ('https://en.wikipedia.org/wiki/Klaus_Krippendorff', 5, 'Klaus Krippendorff'), ('https://en.wikipedia.org/wiki/Radio', 6, 'radio'), ('https://en.wikipedia.org/wiki/Television', 6, 'television'), ('https://en.wikipedia.org/wiki/Key_Word_in_Context', 6, 'Keyword In Context'), ('https://en.wikipedia.org/wiki/Synonym', 6, 'synonyms'), ('https://en.wikipedia.org/wiki/Homonym', 6, 'homonyms')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [165]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,0
1,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,1
2,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,2
3,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,3
4,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,4
5,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,5
6,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,6
7,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,7
8,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,8
9,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [166]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

SyntaxError: can't use starred expression here (<ipython-input-201-3e1350315c6b>, line 1)

In [167]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,"<article> <blockquote class=""messageText Selec...",https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



I won't be continueing from the page I scraped aboved, because it doesn't seem to spider anywhere. But conveniently enough, there's a webpage that serves as a hub of sorts for PUA content, an ostensible "bible" for the community. I'll use that as the starting point of the spidering.

In [170]:
# first, download and soupify the webpage
nexus_url = "http://www.djbible.classicalgasemissions.com"
nexus_raw = requests.get(nexus_url)
nexus_save = 'nexus.html'

with open(nexus_save, mode='w', encoding='utf-8') as f:
    f.write(nexus_raw.text)
with open(nexus_save) as f:
    nexus_soup = bs4.BeautifulSoup(f, 'html.parser')

In [248]:
#create a soup object of all the tables (this is where all the relevant content is)
nexus_content = nexus_soup.body.findAll('table')
other_urls = {"title" : [], "url" :[]}
dj_base_url = 'https://www.sosuave.net/forum'
for index, item in enumerate(nexus_content):
    tagLinks = item.findAll('a', href=re.compile("/forum/"))
    for a_tag in tagLinks:
        reurl = a_tag.get('href')
        link_text = a_tag.text
        other_urls["url"].append(urllib.parse.urljoin(dj_base_url, reurl))
        other_urls['title'].append(link_text)
urls_df = pd.DataFrame(other_urls)

,title,url
0,Most\n commonly\n ...,http://www.sosuave.net/forum/showthread.php?t=...
1,The\n Newbie's\n ...,http://www.sosuave.net/forum/showthread.php?t=...
2,The\n Newbie's\n ...,http://www.sosuave.net/forum/showthread.php?t=...
3,Fifteen\n Lessons,http://www.sosuave.net/forum/showthread.php?t=...
4,THE\n TEN\n ...,http://www.sosuave.net/forum/showthread.php?t=...


In [281]:
#function for scraping text from urls from the spider
def getTextFromThread(thread_title, tar_url):
	#create empty dictionary
	all_dj_texts = {'url':[], "thread":[], "post_no":[], "post": []}

	#scrape webpage
	x_html = requests.get(tar_url)
	x_soup = bs4.BeautifulSoup(x_html.text,'html.parser')

	for index, post in enumerate(x_soup.body.findAll('article')):
		all_dj_texts['url'].append(tar_url)
		all_dj_texts['thread'].append(thread_title)
		all_dj_texts['post'].append(post)
		all_dj_texts['post_no'].append(index)

	return pd.DataFrame(all_dj_texts)

In [286]:
# testing if my functions work
testdf = getTextFromThread(urls_df.iloc[2,0], urls_df.iloc[2,1])
testdf.head()
#they do!

,post,post_no,thread,url
0,"<article> <blockquote class=""messageText Selec...",0,The\n Newbie's\n ...,http://www.sosuave.net/forum/showthread.php?t=...
1,"<article> <blockquote class=""messageText Selec...",1,The\n Newbie's\n ...,http://www.sosuave.net/forum/showthread.php?t=...
2,"<article> <blockquote class=""messageText Selec...",2,The\n Newbie's\n ...,http://www.sosuave.net/forum/showthread.php?t=...
3,"<article> <blockquote class=""messageText Selec...",3,The\n Newbie's\n ...,http://www.sosuave.net/forum/showthread.php?t=...
4,"<article> <blockquote class=""messageText Selec...",4,The\n Newbie's\n ...,http://www.sosuave.net/forum/showthread.php?t=...


In [289]:
#scrape just 5 of the threads
dj_df =  getTextFromThread(urls_df.iloc[0,0], urls_df.iloc[0,1])
for i in range(1,5):
        dj_df.append(getTextFromThread(urls_df.iloc[i,0], urls_df.iloc[i,1]), ignore_index = True)

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [291]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":2964,"posts-type":false,"posts":[{"id":"169306612050","url":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/169306612050","url-with-slug":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/169306612050\/kitty-mind-blown","type":"photo","date-gmt":"2018-01-04 15:00:13 GMT","date":"Thu, 04 Jan 2018 16:00:13","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1515078013,"format":"html","reblog-key":"TZzKhxon","slug":"kitty-mind-blown","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"169306612050\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_169306612050\"><iframe id=\"like_iframe_169306612050\" src=\"http:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=fc298e85f978b8662a643fe0a6b8c638#name=lolcats-lol-cat&amp;post_id=169306612050&amp;co

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [292]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [293]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [ ]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [294]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [295]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [296]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [ ]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [ ]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [ ]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [ ]:
print(readPDF(infoExtractionBytes)[:550])

From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [ ]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [ ]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [ ]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [ ]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [297]:
# download the text

pook_url = "http://www.djbible.classicalgasemissions.com/book_of_pook.pdf"

pook_dl = requests.get(pook_url, stream = True)

In [299]:
#copying the function from above
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString



In [300]:
#import into python

pook_bytes = io.BytesIO(pook_dl.content)

print(readPDF(pook_bytes)[:1000])

 

WWW.SOSUAVE.NET 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

Material by Pook, from the Sosuave.com forums 

Compiled By Gubby 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 

#1 
#2 
#3 
#4 
#5 
#6 
#7 
#8 
#9 
#10 
#11 
#12 
#13 
#14 
#15 

 
Contents 
 
 
 
A Note From The Compiler    
Introduction – A Pook Is A Pook! 
 
Pook Man’s Advice To A Suffering Newbie  
Fifteen Lessons 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

What Every Skinny Guy Should Know 
 
Perfect Is Boring 
 
The Secret Of The Jerk 
 
Looks Or Personality? 
 
Fountain Of Youth 
 
 
Kill That Desperation! 
 
Embrace Your Sexuality! 
 
Embracing Sexuality  
 
Eliminate Desire Or Not 
On Shyness 
 
On The Nature Of Sexuality  
Romanticism – 